In [ ]:
import os, time, requests, urllib, re
from bs4 import BeautifulSoup

In [4]:
#あるディレクトリの下層にあるページURLのリストを取得する関数

def get_links(url, directory):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    links = []

    #1階層下のurlを取得しリストに追加
    for link in soup.find_all('a', href=True):
        href = link['href']
        parsed_href = urllib.parse.urljoin(url, href)
        if url in parsed_href and directory in urllib.parse.urlparse(parsed_href).path:
            links.append(parsed_href)

    # 順序を保持しながら重複を除去
    seen = set()
    unique_links = []
    for link in links:
        if link not in seen:
            unique_links.append(link)
            seen.add(link)

    return unique_links

In [ ]:
#日付けごとにフォルダを作成し、テキストと画像をスクレイピングする関数

def scrape_and_save_content(url_list, save_directory, delay=0.5):
    headers = {'User-Agent': 'Your User Agent Here'}

    for url in url_list:
        time.sleep(delay)  # サーバーへの負荷を減らすための待機時間
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # 日付の取得
        date_tag = soup.find('p', class_='date')
        date_text = date_tag.get_text() if date_tag else '日付け情報なし'
        # 正規表現で日付を抽出し、フォルダ名に適した形式に変換
        match = re.search(r'(\d{4})\.(\d{2})\.(\d{2})', date_text)
        if match:
            date_folder = match.group(1) + match.group(2) + match.group(3)  # YYYYMMDD形式
        else:
            date_folder = '日付け情報なし'

        # 店舗情報の取得
        shop_tag = soup.find('a', class_='shop')
        shop_name = shop_tag.get_text() if shop_tag else '店舗情報なし'
        shop_folder = shop_name.replace('店', '')  # "垂水店" から "垂水" のように変換

        # 日付ごと、店舗ごとのフォルダを作成
        full_path = os.path.join(save_directory, date_folder, shop_folder)
        if not os.path.exists(full_path):
            os.makedirs(full_path)

        # 文章の保存
        text_content = '\n'.join([p.get_text() for p in soup.find_all('p')])
        with open(os.path.join(full_path, f'article_{date_folder}_{shop_folder}.txt'), 'w', encoding='utf-8') as file:
            file.write(text_content)

        # 上から4つの画像のダウンロードと保存
        for i, img in enumerate(soup.find_all('img', src=True)):
            if i >= 4:  # 最初の4つの画像のみを対象
                break
            time.sleep(delay)  # サーバーへの負荷を減らすための待機時間
            image_url = img['src']
            if not image_url.startswith('http'):
                image_url = urllib.parse.urljoin(url, image_url)  # 相対URLを絶対URLに変換
            image_response = requests.get(image_url, headers=headers)
            if image_response.status_code == 200:
                with open(os.path.join(full_path, f'image_{date_folder}_{shop_folder}_{i+1}.jpg'), 'wb') as file:
                    file.write(image_response.content)

In [6]:
#スクレイピングするサイトの設定と確認

url = 'ルートディレクトリを選択'
directory = '対象のディレクトリを選択'
links = get_links(url, directory)

print(links)
print(f'取得URL数: {len(links)}個')

In [7]:
url_list = links
save_directory = '保存先のディレクトリパス'
delay = 2  # リクエスト間の待機時間（秒）
scrape_and_save_content(url_list, save_directory, delay)